In [2]:
#from urllib.request import urlopen, Request
#from harvest import extract_data
import requests
from bs4 import BeautifulSoup
import math
import json
#import csv
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import numpy as np

#USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"

# Helper functions

In [3]:
# if can't load post data in one time, save the loaded list of dictionaries and return later 
def write_list_of_ngram_dicts(list_of_dicts, filename):
    '''store a list of dictionaries into txt file'''
    with open(filename, 'w', encoding='utf-8') as file:
        for dic in list_of_dicts:
            data=json.dumps(dic) 
            file.write(data)
            file.write("\n")

# Get links to posts
and number of replies to each post

In [ ]:
post_links_replies = list()
for p in range(1,1833):  # 1832 pages of posts
    # extract posts on each page
    URL = 'https://www.diabetesdaily.com/forum/forums/type-2-diabetes.14/page-'+str(p)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    # find all posts
    posts = soup.find_all('div', class_ = 'structItem-title')
    # find the corresponding number of pages of each post 
    meta_info = soup.find_all('div', class_ = 'structItem-cell structItem-cell--meta')
    for i in range(len(meta_info)):
        link = 'https://www.diabetesdaily.com' + posts[i].find('a', href = True)['href']
        num_pages = meta_info[i].find('dd').text
        post_links_replies.append([link,num_pages])

# save in a txt file: link + number of replies of each post 
with open("post_links_replies.txt", "w") as f:
    for post in post_links_replies:
        for k in post:
            f.write(k +" ")
        f.write('\n')

In [3]:
# reload post links from txt
with open("post_links_replies.txt") as f:
    post_links_replies = f.readlines()
post_links_replies = [p.strip(' \n').split(' ') for p in post_links_replies]

len(post_links_replies)

18315

# Get post content
post_text, post_link, user_link, likes 
load post links first

In [3]:
# reload post links from txt
with open("post_links_replies.txt") as f:
    post_links_replies = f.readlines()
post_links_replies = [p.strip(' \n').split(' ') for p in post_links_replies]

In [71]:
posts_data = list()

In [106]:
for i in range(len(post_links_replies)):
    link = post_links_replies[i][0]
    
    # some posts are not in the type2diabetes section, will show a '-' for num_pages
    try:
        num_pages = math.ceil((int(post_links_replies[i][1])+1)/20)
    except: # skip this post
        continue
        
    post_data = list()
    for page in range(num_pages):
        url = link + 'page-' + str(page+1)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        replies = soup.find_all('div', class_ = 'message-inner')
        for r in replies:
            r_data = dict()
            r_data['post_text'] = r.find_all('div', 'bbWrapper')[0].text
            r_data['post_link'] = r.find_all('li', class_ = 'u-concealed')[0].find_all('a', href = True)[0]['href']
            try:
                r_data['user_link'] = r.find_all('h4', class_ = 'message-name')[0].find_all('a', href = True)[0]['href']
            except:
                r_data['user_link'] = 'Anonymous'
            # likes
            try: 
                like_link = r.find_all('a', class_ = 'reactionsBar-link', href = True)[0]['href']
                reaction_URL = 'https://www.diabetesdaily.com' + like_link
                like_page = requests.get(reaction_URL)
                like_soup = BeautifulSoup(like_page.content, "html.parser")
                like_people = like_soup.find_all('a', class_ = 'username')
                like_people_link = []
                for p in like_people:
                    like_people_link.append(p['href'])
            except:
                like_people_link = []
            r_data['likes'] = like_people_link
            post_data.append(r_data)

    posts_data.append(post_data)
    
    if i % 1000 == 0:
        # save for every 1000 posts
        write_list_of_ngram_dicts(posts_data, 'post_data.txt')
        
# finally, save all posts
write_list_of_ngram_dicts(posts_data, 'post_data.txt')

In [104]:
# if interrupted during loading, find the latest retrieved post and continue
# print(posts_data[-1][0]['post_link'])
# print(len(posts_data))
# idx = 7096
# print('Try index = ' + str(idx) + ' in post_links_replies: ')
# print(post_links_replies[idx])
# print('Next post:')
# print(post_links_replies[idx+1])

/forum/threads/i-need-to-stop-expecting-perfection.75746/post-721664
7092
Try index = 7096 in post_links_replies: 
['https://www.diabetesdaily.com/forum/threads/i-need-to-stop-expecting-perfection.75746/', '13']
Next post:
['https://www.diabetesdaily.com/forum/threads/is-a-steady-a1c-of-5-9-with-no-meds-better-than-a-lower-a1c-with-meds.72322/', '34']


In [4]:
with open('post_data.txt', 'r', encoding='utf-8') as file:
        data = file.readlines()
posts_data = [json.loads(d) for d in data]
len(posts_data)

18309

In [5]:
posts_data[-1]

[{'post_text': "I posed this in another tread but thought it ought to have a thread of its' own so people who need it can find it.  It's a Pharma Industry initiative and may be useful to some having difficulty with the cost of meds.\n\nhttps://www.pparx.org/Intro.php\n\nALSO, don't forget to check with the manfacturer of your specific medication to see if they have a program from which you may benefit.  They all have them, at some level of assistance, on some drugs.  It's not difficult to find out if you can catch a break from them.",
  'post_link': '/forum/threads/partnership-for-perscription-assistanca-ppa.86/post-347',
  'user_link': '/forum/members/nikkisdad.101/',
  'likes': []},
 {'post_text': 'Thanks for posting that! If any of you find it useful, let me know. I get regular emails asking about prescription assistance and it would be nice to have a few places to send them.',
  'post_link': '/forum/threads/partnership-for-perscription-assistanca-ppa.86/post-349',
  'user_link': '/

# Get user info

In [140]:
# find all distinct users
users = set()
for post in posts_data:
    for reply in post:
        users.add(reply['user_link'])
        for liker in reply['likes']:
            users.add(liker)

users = list(users)
len(users)

8431

In [270]:
users_data = dict()

In [271]:
# scrape to find information of each user
for i in range(len(users)):
    user_data = dict()
    
    if users[i] == 'Anonymous':
        continue
    
    user_url = 'https://www.diabetesdaily.com' + users[i]
    r = requests.get(user_url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    try:
        user_data['username'] = soup.find_all('span', class_ = 'username')[0].text
        user_data['userTitle'] = soup.find_all('span', class_ = 'userTitle')[0].text
    except:
        continue

    # date joined and date last seen
    for k in soup.find_all('dl', class_ = 'pairs pairs--inline'):
        tmp = k.text.strip('\n')
        tmp = re.sub(r'\n{1,}', ':', tmp)
        user_data[tmp.split(':')[0]] = tmp.split(':')[1]

    # num_replies, activity score
    for k in soup.find_all('div', class_ = 'pairJustifier')[0].find_all('dl'):
        tmp = k.text.strip('\n\t')
        tmp = re.sub(r'(\n{1,}|\t{1,})', ':', tmp)
        tmp = re.sub(r':{1,} {0,}', ':', tmp)
        user_data[tmp.split(':')[0]] = tmp.split(':')[1]

    # scrape the subsection
    # about: personal description, followers, followees
    about_url = 'https://www.diabetesdaily.com' + users[i] + 'about'
    r = requests.get(about_url)
    soup_about = BeautifulSoup(r.content, "html.parser")
    # description
    # first section of description
    try:
        d1 = soup_about.find_all('div', class_ = 'bbWrapper')[0].text
    except:
        d1 = ''
    # second section of description
    try:
        d2 = soup_about.find_all('dl', class_ = "pairs pairs--columns pairs--fixedSmall pairs--customField")[0].text.strip('\n\t')
        d2 = re.sub(r'(\n{1,}|\t{1,})', ':', d2)
        d2 = re.sub(r':{1,} {0,}', ':', d2)
        d2 = d2.split(':')[1]
    except: 
        d2 = ''
    if d1 == '':
        d = d2
    elif d2 == '':
        d = d1
    elif (d1 == '') & (d2 == ''):
        d = ''
    else:
        d = d1 + '; ' + d2
    user_data['Description'] = d

    # followers, followees
    followees = []
    followers = []
    try:
        for followee in soup_about.find_all('ul', class_ = "listHeap")[0].find_all('a', href = True):
            followees.append(followee['href'])
        user_data['followees'] = followees
        for follower in soup_about.find_all('ul', class_ = "listHeap")[1].find_all('a', href = True):
            followers.append(follower['href'])
        user_data['followers'] = followers
    except:
        user_data['followees'] = followees
        user_data['followers'] = followers
    
    users_data[users[i]] = user_data
    
    if i % 1000 == 0:
        # save for every 1000 users
        exDict = {'exDict': users_data}
        with open('users_data.txt', 'w') as file:
             file.write(json.dumps(exDict)) # use `json.loads` to do the reverse
        
# finally, save all users
exDict = {'exDict': users_data}
with open('users_data.txt', 'w') as file:
     file.write(json.dumps(exDict)) # use `json.loads` to do the reverse

In [283]:
with open('users_data.txt', 'r', encoding='utf-8') as file:
     exDict = file.readlines()
users_data = json.loads(exDict[0])['exDict']

In [284]:
print('Number of extracted users: ' + str(len(users_data)))
print('Number of total users: ' + str(len(users)))

Number of extracted users: 7646
Number of total users: 8431


## Fix unextracted users
Some user data are not extracted:   
- One possiblity is that they deleted their accounts
- The other possibility is that (for some reason) we need login to see their personal profile

In [ ]:
not_extracted = [k for k in users if k not in users_data] 

# first login using your username and password
URL = 'https://www.diabetesdaily.com/go/login'
driver = webdriver.Chrome(r"C:\Users\Administrator\Desktop\Lab\chromedriver_win32\chromedriver")
driver.get(URL)

username = driver.find_element_by_id("user_login")
password = driver.find_element_by_id("user_pass")

# enter your own username and password
x = input('Enter your username for www.diabetesdaily.com: ')
y = input('Enter your password for www.diabetesdaily.com: ')

username.send_keys(x)
password.send_keys(y)
driver.find_element_by_id("wppb-submit").click()  # can check chrome to see if login successful

In [247]:
users_data2 = dict()

In [262]:
# retrieve the same information from the rest users
for i in range(len(not_extracted))[678:]:
    
    if not_extracted[i] == 'Anonymous':
        continue
        
    user_url = 'https://www.diabetesdaily.com' + not_extracted[i]
    driver.get(user_url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    user_data = dict()
    try: 
        user_data['username'] = soup.find_all('span', class_ = 'is-stroked')[0].text
        user_data['userTitle'] = soup.find_all('span', class_ = 'userTitle')[0].text
    except: 
        if soup.find_all('h1', class_ = 'p-title-value')[0].text == 'Oops! We ran into some problems.': 
            print("i = " + str(i) + ": Can't find page.")
        else:
            print("i = " + str(i) + ": Other issue.")
        continue
            
    # date joined, date last seen
    for k in soup.find_all('dl', class_ = 'pairs pairs--inline'):
        tmp = k.text.strip('\n')
        tmp = re.sub(r'\n{1,}', ':', tmp)
        user_data[tmp.split(':')[0]] = tmp.split(':')[1]

    # num_replies, activity score
    for k in soup.find_all('div', class_ = 'pairJustifier')[0].find_all('dl'):
        tmp = k.text.strip('\n\t')
        tmp = re.sub(r'(\n{1,}|\t{1,})', ':', tmp)
        tmp = re.sub(r':{1,} {0,}', ':', tmp)
        user_data[tmp.split(':')[0]] = tmp.split(':')[1]

    # scrape the subsection
    # about: personal description, followers, followees
    about_url = 'https://www.diabetesdaily.com' + not_extracted[0] + 'about'
    driver.get(about_url)
    soup_about = BeautifulSoup(driver.page_source, 'lxml')

    # description
    # first section of description
    try:
        d1 = soup_about.find_all('div', class_ = 'bbWrapper')[0].text
    except:
        d1 = ''
    # second section of description
    try:
        d2 = soup_about.find_all('dl', class_ = "pairs pairs--columns pairs--fixedSmall pairs--customField")[0].text.strip('\n\t')
        d2 = re.sub(r'(\n{1,}|\t{1,})', ':', d2)
        d2 = re.sub(r':{1,} {0,}', ':', d2)
        d2 = d2.split(':')[1]
    except: 
        d2 = ''

    if d2 == '':
        d = d1
    elif d2 == '':
        d = d1
    elif (d1 == '') & (d2 == ''):
        d = ''
    else:
        d = d1 + '; ' + d2
    user_data['Description'] = d

    # followers, followees
    followees = []
    followers = []
    try:
        for followee in soup_about.find_all('ul', class_ = "listHeap")[0].find_all('a', href = True):
            followees.append(followee['href'])
        user_data['followees'] = followees
        for follower in soup_about.find_all('ul', class_ = "listHeap")[1].find_all('a', href = True):
            followers.append(follower['href'])
        user_data['followers'] = followers
    except:
        user_data['followees'] = followees
        user_data['followers'] = followers
    
    users_data2[not_extracted[i]] = user_data

i = 686: Can't find page.
i = 687: Can't find page.
i = 688: Can't find page.
i = 690: Can't find page.
i = 691: Can't find page.
i = 700: Can't find page.
i = 705: Can't find page.
i = 709: Can't find page.
i = 710: Can't find page.
i = 720: Can't find page.
i = 731: Can't find page.
i = 745: Can't find page.
i = 752: Can't find page.
i = 772: Can't find page.
i = 779: Can't find page.
i = 780: Can't find page.
i = 781: Can't find page.


In [289]:
users_data = {**users_data, **users_data2}
# save
exDict = {'exDict': users_data}
with open('users_data.txt', 'w') as file:
     file.write(json.dumps(exDict))

## Fix follower/followee number 
Some users have more than 12 followers or followees, which are need folded in their profile webpage. We need extra scraping for them.    

In [39]:
# some users have more than 12 followers or followees, need extra scraping
with open('users_data.txt') as file:
     data = file.readlines()
users_data = json.loads(data[0])['exDict']

long_user = [user for user in users_data if (len(users_data[user]['followers']) == 12) | (len(users_data[user]['followees']) == 12)]
len(long_user)

To ensure that we can see everyone's profile, login first. 

In [29]:
# first login using your username and password
URL = 'https://www.diabetesdaily.com/go/login'
driver = webdriver.Chrome(r"C:\Users\Administrator\Desktop\Lab\chromedriver_win32\chromedriver")
driver.get(URL)

username = driver.find_element_by_id("user_login")
password = driver.find_element_by_id("user_pass")

# enter your own username and password
x = input('Enter your username for www.diabetesdaily.com: ')
y = input('Enter your password for www.diabetesdaily.com: ')

username.send_keys(x)
password.send_keys(y)
driver.find_element_by_id("wppb-submit").click()  # can check chrome to see if login successful

Enter your username for www.diabetesdaily.com: 18915993507@163.com
Enter your password for www.diabetesdaily.com: Fcyxx1201..


Then iterate through each user in the _long_user_ list. 

In [41]:
for i in range(len(long_user)):
    # find all the people the current user is following (followees)
    if len(users_data[long_user[i]]['followees']) == 12:
        followee = []
        page = 1
        hasNextPage = True
        while hasNextPage:
            user_url = 'https://www.diabetesdaily.com' + long_user[i] + 'following/page-' + str(page)
            driver.get(user_url)
            soup = BeautifulSoup(driver.page_source, 'lxml')

            # find followee in the current page
            for p in soup.find_all('h3', class_ = 'contentRow-header'):
                followee.append(p.find('a', href = True)['href'])
            # detect if there is a next page 
            if len(soup.find_all('span', class_ = 'block-footer-controls')) > 0:
                page += 1
            else:
                hasNextPage = False

    # find all the followers of the current user
    if len(users_data[long_user[i]]['followers']) == 12:
        follower = []
        page = 1
        hasNextPage = True
        while hasNextPage:
            user_url = 'https://www.diabetesdaily.com' + long_user[i] + 'followers/page-' + str(page)
            driver.get(user_url)
            soup = BeautifulSoup(driver.page_source, 'lxml')

            # find followee in the current page
            for p in soup.find_all('h3', class_ = 'contentRow-header'):
                follower.append(p.find('a', href = True)['href'])
            # detect if there is a next page 
            if len(soup.find_all('span', class_ = 'block-footer-controls')) > 0:
                page += 1
            else:
                hasNextPage = False
    
    users_data[long_user[i]]['followees'] = followee
    users_data[long_user[i]]['followers'] = follower

In [59]:
# save
exDict = {'exDict': users_data}
with open('users_data.txt', 'w') as file:
     file.write(json.dumps(exDict))

# Interaction matrix
Store each interaction between a user $u$ and a post $i$, defined as $u$ replying to $i$, in a csv file.    

In [5]:
import pandas as pd
import json
with open('post_data.txt', 'r', encoding='utf-8') as file:
        data = file.readlines()
posts_data = [json.loads(d) for d in data]

with open('users_data.txt', 'r', encoding='utf-8') as file:
     exDict = file.readlines()
users_data = json.loads(exDict[0])['exDict']

In [9]:
# only save interactions of users that we have data
interactions = []
for i in range(len(posts_data)):
    for reply in posts_data[i]:
        if reply['user_link'] in users_data.keys():
            interactions.append((i, reply['user_link']))

df = pd.DataFrame(interactions)
df.columns = ['post_id', 'user_link']
df = df.value_counts().reset_index()
df.columns = ['post_id', 'user_link', 'num_replies']

# some users have no interaction data, need to include them 
exist_users = df['user_link'].unique()
for user in users_data:
    if user not in exist_users:
        df = df.append(pd.DataFrame({'post_id': ['no_post'], 'user_link': user, 'num_replies': 0}), ignore_index = True)

df.to_csv('user_network/interactions.csv', index = False)